In [2]:
from box import Box
from rich import print as rprint


# Create a nested Box object for better demonstration
# Add a list to the variable

a = Box({
    "length": 10,
    "width": 5,
    "height": 3,
    "color": "blue",
    "meta": {
        "owner": "Alice",
        "location": {"city": "Paris", "country": "France"}
    },
    "tags": ["storage", "blue", {"nam":{"large":0,"a":9}}]
})

rprint(a.to_yaml())

length: 10
width: 5
height: 3
color: blue
meta:
  owner: Alice
  location:
    city: Paris
    country: France
tags:
- storage
- blue
- nam:
    large: 0
    a: 9

In [3]:
from box import Box

def merge_boxes(a: Box, b: Box, unique_strings_in_lists: bool = True) -> Box:
    """
    Deep merge two Box objects:
    - b overrides a.
    - Dicts/Boxes are merged recursively.
    - Lists:
        * Single-key dicts/Boxes with same key → merge recursively
        * Strings → append only if not already present (controlled by unique_strings_in_lists)
        * Other elements → append
    - Scalars/other values: overwritten by b
    """
    result = Box(a.to_dict())

    for key, b_val in b.items():
        if key not in result:
            result[key] = b_val
            continue

        a_val = result[key]

        # Case 1: Both dicts/Boxes → merge recursively
        if isinstance(a_val, (Box, dict)) and isinstance(b_val, (Box, dict)):
            result[key] = merge_boxes(Box(a_val), Box(b_val), unique_strings_in_lists)

        # Case 2: Both lists → merge elements
        elif isinstance(a_val, list) and isinstance(b_val, list):
            # Build id_map for single-key dicts/Boxes
            id_map = {next(iter(item)): i for i, item in enumerate(a_val)
                      if isinstance(item, (Box, dict)) and len(item) == 1}

            for b_item in b_val:
                if isinstance(b_item, (Box, dict)) and len(b_item) == 1:
                    b_id = next(iter(b_item))
                    if b_id in id_map:
                        idx = id_map[b_id]
                        a_val[idx] = merge_boxes(Box(a_val[idx]), Box(b_item), unique_strings_in_lists)
                    else:
                        a_val.append(b_item)
                elif isinstance(b_item, str) and unique_strings_in_lists:
                    if b_item not in a_val:
                        a_val.append(b_item)
                else:
                    a_val.append(b_item)

        # Case 3: Otherwise → overwrite
        else:
            result[key] = b_val

    return result


In [6]:
# Create another Box object
b2 = Box({
    "length": 20,
    "width": 8,
    "material": "wood",
    "meta": {
        "owner": "Bob",
        "location": {"city": "Berlin"}
    },
    "tags": ["new", "wooden"]
})

# Merge b2 into a, overwriting in case of collision
#c= Box(a)
#c.merge_update(b2)
c= a + b2  # This will concatenate lists instead of overwriting
d = merge_boxes(a, b2)  # This will do a deep merge with custom logic
rprint(c.to_yaml())
rprint(d.to_yaml()) 

length: 20
width: 8
height: 3
color: blue
meta:
  owner: Bob
  location:
    city: Berlin
    country: France
tags:
- new
- wooden
material: wood

length: 20
width: 8
height: 3
color: blue
meta:
  owner: Bob
  location:
    city: Berlin
    country: France
tags:
- storage
- blue
- nam:
    large: 0
    a: 9
- new
- wooden
material: wood

In [29]:
tempbox=Box(default_box=True,box_dots=True)

tempbox["a.e.f"]=11
rprint(tempbox.to_yaml())


a:
  e:
    f: 11